In [36]:
import psycopg2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib  

conn = psycopg2.connect(
    dbname="branchdsprojectgps",
    user="datascientist",
    password="47eyYBLT0laW5j9U24Uuy8gLcrN",
    host="branchhomeworkdb.ccc0r2wfuew7.us-east-1.rds.amazonaws.com",
    port="5432"
)

loan_outcomes_df = pd.read_sql("SELECT * FROM loan_outcomes", conn)
user_attributes_df = pd.read_sql("SELECT * FROM user_attributes", conn)

conn.close()

merged_df = pd.merge(loan_outcomes_df, user_attributes_df, on="user_id", how="inner")

merged_df['loan_outcome'] = merged_df['loan_outcome'].map({'defaulted': 0, 'repaid': 1})

X = merged_df[['age', 'cash_incoming_30days']]
y = merged_df['loan_outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

joblib.dump(model, 'model.pkl')

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")
print(classification_report(y_test, y_pred))


Accuracy: 52.50%
              precision    recall  f1-score   support

           0       0.48      0.43      0.46        37
           1       0.55      0.60      0.58        43

    accuracy                           0.53        80
   macro avg       0.52      0.52      0.52        80
weighted avg       0.52      0.53      0.52        80

